In [5]:
%cd /content/drive/MyDrive/DL_HW3

/content/drive/MyDrive/DL_HW3


In [ ]:
!unzip test_images.zip -d /content/drive/MyDrive/DL_HW3

In [ ]:
!unzip train_images.zip -d /content/drive/MyDrive/DL_HW3

In [6]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
#!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.7.0+cu101)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.8.1+cu101)
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-jvnjhodr
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-jvnjhodr
  Created wheel for fvcore: filename=fvcore-0.1.2-cp36-none-any.whl size=48498 sha256=8dd721fb5d9f579cf712716c7f4a3dd5f4d34250cd894ee7fdf28ac95b6693d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-notn13vv/wheels/48/53/79/3c6485543a4455a0006f5db590ab9957622b6227011941de06
Successfully built fvcore
Obtaining file:///content/drive/MyDrive/DL_HW3/detectron2_repo
  Found existing installation: detectron2 0.3
    Can't uninstall 'detectron2'. No files were found to uninstall.
  Running setup.py develop for detectron2


In [7]:
%cd /content/drive/MyDrive/DL_HW3/detectron2_repo

/content/drive/MyDrive/DL_HW3/detectron2_repo


In [8]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("Tiny_dataset", {}, "/content/drive/MyDrive/DL_HW3/pascal_train.json", "/content/drive/MyDrive/DL_HW3/train_images")

In [9]:
from detectron2.data import MetadataCatalog, DatasetCatalog
Tiny_metadata = MetadataCatalog.get("Tiny_dataset")

In [10]:
Tiny_metadata

Metadata(evaluator_type='coco', image_root='/content/drive/MyDrive/DL_HW3/train_images', json_file='/content/drive/MyDrive/DL_HW3/pascal_train.json', name='Tiny_dataset')

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file("./configs/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("Tiny_dataset",)
cfg.DATASETS.TEST = ()  
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-101.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = (
    15000
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 20  

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)  # If want resume training, set resume = true
trainer.train()

[12/02 14:36:13 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:
import detectron2
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from itertools import groupby
from pycocotools import mask as maskutil

from pycocotools.coco import COCO
import json
setup_logger()

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    compressed_rle = maskutil.frPyObjects(rle, rle.get('size')[0], rle.get('size')[1])
    compressed_rle['counts'] = str(compressed_rle['counts'], encoding='utf-8')
    return compressed_rle

cfg = get_cfg()
cfg.merge_from_file("./configs/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("Tiny_dataset", )
cfg.DATASETS.TEST = ()
cfg.SOLVER.MAX_ITER = 500
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 20 
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/DL_HW3/detectron2_repo/output/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.55

predictor = DefaultPredictor(cfg)

cocoGt = COCO("/content/drive/MyDrive/DL_HW3/test.json")


coco_dt = []
count=0

for imgid in cocoGt.imgs:
    image = cv2.imread("/content/drive/MyDrive/DL_HW3/test_images/" + cocoGt.loadImgs(ids=imgid)[0]['file_name'])[:,:,::-1] 
    result = predictor(image) # run inference
    for i in range(len(result["instances"].scores)):
        pred = {}
        pred['image_id'] = imgid
        pred['category_id'] = int(result["instances"].pred_classes[i] + 1)
        binary_mask = result["instances"].pred_masks[i].to("cpu").numpy()
        pred['segmentation'] = binary_mask_to_rle(binary_mask) # save binary mask to RLE
        pred['score'] = float(result["instances"].scores[i])
        coco_dt.append(pred)
    
    # Visualize the inference result
    if(count<10):
      v = Visualizer(image,
                metadata=Tiny_metadata,
                scale=1.2, 
                instance_mode=ColorMode.IMAGE_BW  
      )
      v = v.draw_instance_predictions(result["instances"].to("cpu"))
      cv2_imshow(v.get_image()[:,:,::-1])

    count+=1
    

with open("/content/drive/MyDrive/DL_HW3/detectron2_repo/output/309553007.json", "w") as f:
    json.dump(coco_dt, f)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


/content/drive/MyDrive/DL_HW3/detectron2_repo/detectron2/modeling/roi_heads/fast_rcnn.py:124: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()
